<img src="https://iteso.mx/documents/27014/202031/Logo-ITESO-MinimoH.png"
     align="right"
     width="300"/>

# Redes Neuronales Convulucionales (CNN) multivariadas para series de tiempo
## *Modelos no lineales para pronósitico*  - Pedro Martinez

---

# CNN 1D multivariadas para pronóstico del clima

Hoy en día, Google y OpenWeather usan variantes de CNN en sus pipelines de nowcasting meteorológico ya que son capaces de detectar patrones temporales locales en series de tiempo, como picos de temperatura o tendencias estacionales.

Usaremos datos reales de la API [Open-Meteo](https://open-meteo.com/) para **predecir la temperatura de las próximas 48 h**, empleando las variables:

| Variable | Descripción | Unidad | Fuente |
|-----------|--------------|---------|---------|
| **Temperatura** | Temperatura del aire a 2 metros | °C (grados Celsius) | `temperature_2m` |
| **Humedad relativa** | Porcentaje de saturación de vapor de agua | % | `relative_humidity_2m` |
| **Nubosidad** | Cobertura nubosa promedio | % | `cloud_cover` |
| **Velocidad del viento** | Velocidad del viento a 10 metros | m/s | `wind_speed_10m` |

---

Una CNN aplicada a series de tiempo multivariables procesa una secuencia de observaciones:

$$
X_t =
\begin{bmatrix}
x_{t,1} & x_{t,2} & \dots & x_{t,n}
\end{bmatrix}
\in \mathbb{R}^{n}
$$

donde $n$ es el número de variables de entrada (en este caso, 4).

Si utilizamos una **ventana temporal de tamaño $T$**, la entrada al modelo es:

$$
\mathbf{X} =
\begin{bmatrix}
X_{t-T+1} \\
X_{t-T+2} \\
\vdots \\
X_t
\end{bmatrix}
\in \mathbb{R}^{T \times n}
$$

Cada capa convolucional aplica un conjunto de filtros (*kernels*) que aprenden patrones espacio-temporales en la ventana:

$$
h_j^{(l)}(t) = f \!\left(
\sum_{i=1}^{n}
\sum_{k=0}^{K-1}
w_{ij}^{(l)}(k) \, x_i(t - k)
+ b_j^{(l)}
\right)
$$

donde:

- $w_{ij}^{(l)}(k)$ son los pesos del filtro $j$ en la posición $k$ de la capa $l$.  
- $b_j^{(l)}$ es el sesgo asociado al filtro $j$.  
- $f(\cdot)$ es una función de activación no lineal (típicamente **ReLU**).  
- $K$ es el tamaño del kernel (por ejemplo, 3 horas).

El resultado de esta capa convolucional se **aplasta (*flatten*)** y se conecta a una capa **densa (fully connected)** para generar la predicción final.

---

## Interpretación

- La CNN aprende **patrones locales** (por ejemplo, cómo la humedad y la nubosidad afectan la temperatura en las últimas horas).  
- Al combinar varias variables de entrada, la red actúa como un **modelo multivariable no lineal**.  
- El entrenamiento minimiza una función de pérdida (MSE):


In [14]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import requests
import pandas as pd
import plotly.express as px
import tensorflow as tf
from tensorflow.keras import layers, models, Input
import plotly.graph_objects as go
from datetime import datetime, timedelta, timezone
import pytz


In [19]:
CITY_TZ = "America/Mexico_City"
lat, lon = 20.31, -103.18  # Chapala

tz = pytz.timezone(CITY_TZ)

# --- API Open-Meteo ---
url = (
    "https://api.open-meteo.com/v1/forecast"
    f"?latitude={lat}&longitude={lon}"
    "&hourly=temperature_2m,relative_humidity_2m,cloud_cover,wind_speed_10m"
    "&past_days=15"
    f"&timezone={CITY_TZ}"
)
res = requests.get(url).json()

# --- Construcción del DataFrame ---
df = pd.DataFrame({
    "time": pd.to_datetime(res["hourly"]["time"]),
    "temp": res["hourly"]["temperature_2m"],
    "humidity": res["hourly"]["relative_humidity_2m"],
    "clouds": res["hourly"]["cloud_cover"],
    "wind": res["hourly"]["wind_speed_10m"],
}).set_index("time")

# 🔹 Asegurar zona local
df.index = df.index.tz_localize(tz)

# 🔹 Calcular hora local actual y filtrar HASTA una hora atrás
now_local = datetime.now(tz)
cutoff = now_local - timedelta(hours=1)
df = df[df.index <= cutoff]


In [20]:
fig = px.line(df, y=["temp","humidity","clouds","wind"],
              title="Variables meteorológicas",
              template="plotly_white")
fig.show()

En equipos de 2 personas, realizar las siguientes partes:

Seleccionar variable objetivo

Escalar los datos

Crear ventanas temporales


*   Se sugiere utilizar 12 o 24 valores anteriores


Dividir en entrenamiento y prueba

Construir la CNN en Keras
- Recordando las recomendaciones de la clase pasada
- Cambiar número de filtros (16, 32, 64)
- Cambiar tamaño del kernel (2, 3, 5)
- Cambiar función de activación (relu, tanh)
- Cambiar batch_size o epochs

Entrenar el modelo utilizando una semilla para reproducibilidad

Evaluar y visualizar utilizando plotly